In [64]:
import csv
import re
import json
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
from collections import OrderedDict
import itertools

In [45]:
def create_friends_dict():
    # this dictionnary contains every user's list of friends using user IDs
    friends_dict={}
    with open('data/friends.csv', 'r') as file:
        csvreader = csv.reader(file)
        for line in csvreader:
            if len(line) == 2:
                if line[0] in friends_dict:
                    # append the new number to the existing array at this slot
                    friends_dict[line[0]].append(line[1])
                else:
                    # create a new array in this slot
                    friends_dict[line[0]] = [line[1]]
        return friends_dict

friends_dict = create_friends_dict()


print(friends_dict['360'])


['4719', '4720', '4721', '4722', '14', '4723', '1020', '4724', '4725', '4726', '4727', '4728', '4729', '1286', '4730', '4731', '4732', '4733', '4734', '4735', '4736', '4737', '1075']


In [4]:
def create_concerts_amount_dict():
    #this dictionnary contains the number of concerts for one genre
    concert_numbers={}
    with open('data/n_concerts.txt', 'r') as f:
        lines = f.readlines()
        for l in lines:
            x=re.search('([\w| |/|&|-]+):([0-9]+)',l)
            if x:
                concert_numbers[x.group(1)]=[x.group(2)][0]
    return concert_numbers

concert_numbers = create_concerts_amount_dict()
# call e.g: print('number of classical concerts: ',concert_numbers['Classical'])

In [6]:
with open('data/preferences.json', 'r') as f:
  preferences_data = json.load(f)

In [7]:
with open('data/preferences_description.txt') as f:
  pref_des_data_lines = f.readlines()

In [9]:
# get the genres a user likes using their ID
def genres_of_id(id):
  genres_num=[]
  genres=[]
  preferences=preferences_data[str(id)]
  for i in range(len(preferences)):
    if preferences[i]=='1':
      genres_num.append(i)
  for num in genres_num:
    genres.append(pref_des_data_lines[num+2].rstrip())
  return(genres)
  
genres_of_id(1)
# call: genres_of_id(1612)

['Indie Rock/Rock pop', 'Baroque', 'Nursery Rhymes', 'Hard Rock']

In [61]:
# calculates the number of concert two friends (id1,id2) are attending together
def concerts_of_friends(id1,id2):
  total_number_of_concert=0
  # calculates the number of concert for every genre they both like
  genres_both_like= [element for element in genres_of_id(id1) if element in genres_of_id(id2)]
  for g in genres_both_like:
    total_number_of_concert+=0.393*float(concert_numbers[g])
  # calculates the number of concert for every genre only one of them likes
  genres_one_likes=list(set(genres_of_id(id1)).symmetric_difference(genres_of_id(id2)))
  for g in genres_one_likes:
    total_number_of_concert+=0.018*float(concert_numbers[g])
  genres=concert_numbers.keys()
  # calculates the number of concert for every genre none of them likes
  genres_none_like=[element for element in genres if element not in list(set(genres_both_like)|set(genres_one_likes))]
  for g in genres_none_like:
    total_number_of_concert+=0.002*float(concert_numbers[g])
  return total_number_of_concert

concerts_of_friends(4,4)
# call: concerts_of_friends(360,4719)

33.91200000000002

In [42]:
def all_concerts_of_user(id):
  res=0
  for friend in friends_dict[str(id)]:
    res += concerts_of_friends(id,int(friend))
  return res
  
all_concerts_of_user(360)

212.222

In [47]:
def get_concert_number(concert):
  #retrieving number of concerts
  #this dictionnary contains the number of concerts for one genre
  concert_numbers={}
  with open('data/n_concerts.txt', 'r') as f:
      lines = f.readlines()
      for l in lines:
        x=re.search('([\w| |/|&|-]+):([0-9]+)',l)
        if x:
          concert_numbers[x.group(1)]=[x.group(2)][0]

  return concert_numbers[concert]
print(get_concert_number('Classical'))

24


In [51]:
def create_textfile_of_percentage_with(unsorted_dict, type):

    # sorted dict of useres and number of concerts they go to
    weights_of_users_list = sorted(unsorted_dict.items(), key=lambda x:x[1], reverse=True)

    # 12% threshold
    twelve_tresh = int(len(weights_of_users_list)*0.12)
    # create a string of users
    long_string = ""
    for counter in range (twelve_tresh):
        user = weights_of_users_list[counter]
        long_string += str(user[0])+"\n"

    # write into txt file
    filename = type + '_team_8.txt'
    with open(filename, 'w') as f:
        f.write(long_string)
    f.close()



In [72]:
def create_network():
    # create an empty graph
    G = nx.Graph()
    terminator = len(friends_dict.keys()) / 1

    # add nodes to the graph representing individual people
    for counter, user in enumerate(friends_dict.keys()):
        if counter == int(terminator):
            print(int(terminator))
            break
        G.add_node(user)
    
    nodes = G.nodes()
    for node, friend in itertools.combinations(nodes, 2):
        if friend == node:
            print("skip")
            continue
        if not G.has_edge(node,friend):
            weight = concerts_of_friends(int(node),int(friend))
            G.add_edge(user, friend,weight=weight)
    # add edges to the graph representing co-visits to concerts
    # for user in friends_dict.keys():
    #     for friend in friends_dict[user]:
    #         if not G.has_edge(user,friend):
    #             weight = concerts_of_friends(int(user),int(friend))
    #             G.add_edge(user, friend,weight=weight)
    print('num of nodes',G.number_of_nodes())
    print('num of edges',G.number_of_edges())
    # add edge key attribute
    closeness_centrality = nx.closeness_centrality(G, distance= 'weight')

    return closeness_centrality

closeness_centrality = create_network()
print(closeness_centrality)


num of nodes 6717
num of edges 6716
{'360': 0.0, '4719': 0.04970047689877755, '4720': 0.049660986104570835, '4721': 0.042774286852173216, '4722': 0.0773722439987498, '14': 0.08505910123745856, '4723': 0.051895398210047655, '1020': 0.0732894350901792, '4724': 0.07363482096614007, '4725': 0.04743713466216746, '4726': 0.04323664096268499, '4728': 0.06301780401127496, '4729': 0.07596168576941395, '1286': 0.0773722439987498, '4730': 0.08683169951486437, '4731': 0.08683169951486437, '4732': 0.08448420917809256, '4733': 0.051715583560266246, '4734': 0.08448420917809256, '4735': 0.08683169951486437, '4736': 0.08505910123745856, '4737': 0.0796398276289817, '1075': 0.0811916897163041, '5375': 0.07095957218394719, '5100': 0.07095957218394719, '5880': 0.08587722081184555, '3203': 0.061377816478816585, '1172': 0.019893161543941055, '1110': 0.06984932114865466, '1173': 0.08471323094335476, '5607': 0.08471323094335476, '4093': 0.04846245291709762, '4043': 0.0467627739081927, '4653': 0.049114631960297

In [ ]:
def create_naive_userlist():
    # user concerts with friends sorted
    weights_of_users = OrderedDict({})
    # fill dict
    for user in friends_dict.keys():
        weights_of_users[user] = all_concerts_of_user(int(user))
        
    create_textfile_of_percentage_with(weights_of_users, 'naive')


In [71]:
def create_centrality_userlist():
    # user centrality degree sorted
    create_textfile_of_percentage_with(closeness_centrality, 'centrality')
